# Homework 7/8

### Snow Hydrology

### Clinton Alden

In [6]:
import pandas as pd
from metloom.pointdata import SnotelPointData
import altair as alt
import numpy as np
from metpy.units import units
import numpy as np
import altair as alt
alt.data_transformers.disable_max_rows()
import matplotlib.pyplot as plt
import datetime as dt

# Problem 1 
Temperature-Index Method For this problem, we will use the Temperature-Index method for predicting snow melt, discussed in Module 7 and introduced in Lab 7-3.

## a)
Calculate Mf using measurements from the Schofield Pass Snotel station and report your value. I.E. mimic Lab 7-3 but for measurements from a different Snotel station. Use the same date range, 1990-2024. Note that we downloaded and used Schofield pass data in Lab 2-1.


In [7]:
start_date = '1990-01-01'
end_date = '2024-01-10'

snotel_point_butte = SnotelPointData("380:CO:SNTL", "Butte")
SNOTEL_VARS = [
    snotel_point_butte.ALLOWED_VARIABLES.SWE,
    snotel_point_butte.ALLOWED_VARIABLES.TEMPMAX,
    snotel_point_butte.ALLOWED_VARIABLES.PRECIPITATION
]
df_butte_longterm = snotel_point_butte.get_daily_data(
    pd.to_datetime(start_date), pd.to_datetime(end_date),
    SNOTEL_VARS
)
df_butte_longterm = df_butte_longterm.reset_index()
# df_butte_longterm.to_csv('../data/butte_longterm.csv')

In [9]:
df_schofield_longterm = pd.read_csv('/Users/clintonalden/Documents/grad_school/cewa568_W25/data/schofield_longterm.csv')
df_butte_longterm = pd.read_csv('/Users/clintonalden/Documents/grad_school/cewa568_W25/data/butte_longterm.csv')

df = df_schofield_longterm
def melt_rate(df):
    df['datetime'] = pd.to_datetime(df['datetime'])
    df['year'] = df['datetime'].dt.year
    df['date_no_year'] = df['datetime'].apply(
        lambda dt: dt.replace(year=1999) if dt.month in [10,11,12] else dt.replace(year=2000)
    )

    delta_swe = df.groupby('year').apply(lambda df: df.set_index('datetime')[['SWE']].diff())
    delta_swe = delta_swe.rename(columns={'SWE': 'Delta SWE'})

    delta_swe = delta_swe[delta_swe.index.get_level_values(1).month > 3] 
    delta_swe = delta_swe[delta_swe.index.get_level_values(1).month < 7]
    delta_swe = delta_swe[delta_swe['Delta SWE'] < 0]
    delta_swe['MELT RATE'] = - delta_swe['Delta SWE']

    delta_swe = delta_swe.reset_index().set_index('datetime').join(
        df.set_index('datetime')[['MAX AIR TEMP', 'PRECIPITATION']]
    )

    delta_swe['Delta SWE'] = (delta_swe['Delta SWE'].values * units('inches')).to(units('millimeters')).magnitude
    delta_swe['MAX AIR TEMP'] = (delta_swe['MAX AIR TEMP'].values * units('degF')).to(units('degC')).magnitude
    delta_swe['PRECIPITATION'] = (delta_swe['PRECIPITATION'].values * units('inches')).to(units('millimeters')).magnitude
    delta_swe['MELT RATE'] = (delta_swe['MELT RATE'].values * units('inches')).to(units('millimeters')).magnitude

    chart_scatter = alt.Chart(delta_swe).mark_point(size=5).encode(
        alt.X('MAX AIR TEMP:Q').title('Max daily air temp. (˚C)'),
        alt.Y('MELT RATE:Q').title('Melt rate (mm/day)')
    )
    chart_2d_histogram = alt.Chart(delta_swe).mark_rect().encode(
        alt.X('MAX AIR TEMP:Q', bin=alt.Bin(maxbins=30)).title('Max daily air temp. (˚C)'),
        alt.Y('MELT RATE:Q', bin=alt.Bin(maxbins=30)).title('Melt rate (mm/day)'),
        alt.Color('count()').title('Point Density')
    )
    (chart_scatter | chart_2d_histogram).resolve_scale(x='shared', y='shared')

    # filter data points
    src = delta_swe[delta_swe['MAX AIR TEMP'] < 25]

    # Perform linear regression with zero intercept
    x = src['MAX AIR TEMP']
    y = src['MELT RATE']
    DEGREE_DAY_FACTOR = np.sum(x * y) / np.sum(x * x)

    # Create the plot for the line of best fit
    # Line of best fit using the slope (DEGREE_DAY_FACTOR) and zero intercept
    line = alt.Chart(src).mark_line(color='red').encode(
        alt.X('MAX AIR TEMP:Q'),
        alt.Y('MELT RATE:Q')
    ).transform_calculate(
        "MELT RATE", f"{DEGREE_DAY_FACTOR} * datum['MAX AIR TEMP']"
    )

    # Add it to the charts above.
    # Add the estimated degree day factor (the DEGREE_DAY_FACTOR of the line of best fit)
    #    to the charts.
    (
        (chart_scatter + line).properties(title=f'M_f = {round(DEGREE_DAY_FACTOR,2)} mm/˚C/day')  
        | 
        (chart_2d_histogram + line).properties(title=f'M_f = {round(DEGREE_DAY_FACTOR,2)} mm/˚C/day')
    ).resolve_scale(x='shared', y='shared')

/var/folders/db/7c91t8d577n1zsyhth1xc7840000gn/T/ipykernel_23298/480499881.py:11: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  delta_swe = df.groupby('year').apply(lambda df: df.set_index('datetime')[['SWE']].diff())


alt.HConcatChart(...)

## b)
Compare Mf values calculated at the Butte and Schofield Pass Snotel stations. Explain why you think one is larger/smaller than the other.

## c)
 Choose an Mf value calculated from either the Butte or Schofield Pass Snotel stations, and model snowpack ablation at Kettle Ponds. I.E. mimic the second half of Lab 7-3 where we modeled snowpack ablation for the Butte Snotel station, but model snowpack ablation at Kettle Ponds using air temperature measurements at Kettle Ponds. Your solution to this problem should include:

-A statement of which Mf value you selected to model snowpack ablation at Kettle Ponds, and why.

-A plot of modeled and measured SWE during the ablation period (April 1 - June 1) (I.E. create a similar plot to the final plot generated in Lab 7-3).

## d)

Explain why you think modeled and measured ablation from part c disagree. What do you know happened during the 2024 ablation period and how are those represented or not represented in the degree day factor, Mf.

## e)

We know that the melting of the snowpack is a result of the snowpack energy balance (see Module 4). Explain, for each term in the energy balance equation, if and how the temperature-index method incorporates the influence of that energy flux term. To help you get started, here's one part of the answer, for the ground heat flux term: The ground heat flux is not represented in the temperature-index method, because the air temperature and the ground heat flux are not related. First of all, the ground heat flux out of the earth is approximately constant, Second, snow is a great insulator, so ground heat flux is unlikely to impact air temperature. Therefore, air temperature does not incorporate the influence of the ground heat flux on snowpack melt.



# Problem 2

## a) 
Lab 8-2 demonstrated that the model of the snowpack in the upper East River Valley melted out about three weeks later than the actual snowpack at Kettle Ponds. Explain, using the two plots generated in Lab 8-2 (modeled vs. measured SWE and modeled vs. measured energy balance), why you think the modeled snowpack melted out later than the measured snowpack.

## b)

Lab 8-2 demonstrated how the timing of snowmelt propagated spatially throughout the basin (see the 2x2 grid of snow melt plots towards the end of the lab). Explain how the spatial distribution of melt rates propagates throughout the basin, making sure to include the details of where melt starts first and why, and where it happens last and why. Explain why this spatio-temporal relationship exists.